Import Libraries

In [1]:
import pickle
import featuretools as ft
import time
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from featuretools.selection import (
    remove_highly_correlated_features,
    remove_highly_null_features,
    remove_single_value_features,
)
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
import warnings
import traceback
from Dilshan import DilFeat

Get Data

In [2]:
# Paths to the pickle files
independent_path = '../Data/Cleaned Data/Independent_Data_dictionary.pkl'
dependent_path = '../Data/Cleaned Data/Dependent_Data_dictionary.pkl'

# Load the data
with open(independent_path, 'rb') as f:
    Independent_Data = pickle.load(f)

with open(dependent_path, 'rb') as f:
    Dependent_Data = pickle.load(f)

Encode the dependent variable

In [3]:
Dependent_Data_Encoded = {}

for name, df in Dependent_Data.items():
    # Creating a copy of the dataframe to modify
    encoded_df = df.copy()
    
    # Initialize the LabelEncoder
    le = LabelEncoder()
    
    # Iterate through each column in the dataframe
    for column in df.columns:
        # Encode the column if it is of type 'object' or 'category'
        if df[column].dtype == 'object' or df[column].dtype.name == 'category':
            # Fit label encoder and return encoded labels
            encoded_df[column] = le.fit_transform(df[column])
    
    # Add the encoded dataframe to the new dictionary
    Dependent_Data_Encoded[name] = encoded_df

# Now Dependent_Data_Encoded contains all the label-encoded dataframes


Applying Feature Tools

In [4]:
# Suppress warnings for metrics that might be undefined for some classifiers
warnings.filterwarnings("ignore")

# Initialize cross validator
kf = KFold(n_splits=5, shuffle=True, random_state=42)

#Initiate the model to normalize data
scaler = MinMaxScaler()

In [ ]:
# Instantiate the class
dil_feat = DilFeat()

# Processing datasets
for dataset_name, X in Independent_Data.items():
    try:
        fold_data = {}  # Dictionary to store fold data
        times = []
        print(f'Processing dataset: {dataset_name}')
        y = Dependent_Data_Encoded[dataset_name]
        
        fold_count = 1
        # Manually handling cross-validation to incorporate FT
        for train_index, test_index in kf.split(X):
            fold_key = f'fold{fold_count}'
            fold_count += 1
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]
            
            # FT feature engineering
            # Start timing for Feature Tools
            start_time = time.time()
            # Transform the data
            X_train_transformed, X_test_transformed = dil_feat.transform(X_train, X_test)       
            # End timing after fitting the model
            transform_time = time.time() - start_time
            times.append(transform_time)

            # Apply normalization on each DataFrame
            X_train_transformed = pd.DataFrame(scaler.fit_transform(X_train_transformed), columns=X_train_transformed.columns, index=X_train_transformed.index)
            X_test_transformed = pd.DataFrame(scaler.transform(X_test_transformed), columns=X_test_transformed.columns, index=X_test_transformed.index)
            
            fold_data[fold_key] = {
                'Training_Independent': X_train_transformed,
                'Training_Dependent': y_train,
                'Testing_Independent': X_test_transformed,
                'Testing_Dependent': y_test,
                'Timing': transform_time
            }

        # Serialize fold data to a .pkl file
        with open(f'../Data/FT/{dataset_name}_fold_data.pkl', 'wb') as file:
            pickle.dump(fold_data, file)

    except Exception as e:
        print(f"Error processing {dataset_name}: {e}")
        traceback.print_exc()  # This will print the traceback of the exception


